# 📋 Data Cleaning: Best Selling Phones

## 🛠️ 1. Import libraries and load raw data

In [19]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import datetime
from datetime import datetime

# Load the raw dataset
file = 'product_2025_05_07.csv'
df = pd.read_csv(file)

## 🛠️ 2. Clean missing values

In [20]:
# Replace missing values with 'NaN'
list_na=dict(df.isna().sum())
for col,count in list_na.items():
    if count > 0:
        df.loc[df[col].isna(),col] = 'NaN'


## 🛠️3. Clean text fields (Utility, Screen, Rating)

In [21]:
# Clean text fields by removing newline characters and extra spaces

def clean_utility(row):
    return ' '.join(row.replace('\n','').strip(' ').split())
df['phone_utility']=df['phone_utility'].apply(clean_utility)

def clean_screen(row):
    _=row.replace('\n','').strip().split()
    return ' '.join(_)
df['phone_screen']=df['phone_screen'].apply(clean_screen)

def clean_rating(row):
    _=row.replace('\n','').strip().split()
    return ' '.join(_)
df['phone_rating']=df['phone_rating'].apply(clean_rating)


In [22]:
df

,phone_id,phone_name,phone_price,phone_brand,category,phone_img,phone_screen,phone_utility,phone_rating
0,323543,Điện thoại OPPO A60 8GB/128GB,5490000.0,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/323543/...,"HD+ 6.67""",Chip Snapdragon 680 RAM: 8 GB Dung lượng: 128 ...,"4.9 • Đã bán 230,2k So sánh"
1,329149,Điện thoại iPhone 16 Pro Max 256GB,30990000.0,iPhone (Apple),Điện thoại,https://cdn.tgdd.vn/Products/Images/42/329149/...,"Super Retina XDR 6.9""",Chip Apple A18 Pro 6 nhân RAM: 8 GB Dung lượng...,"4.9 • Đã bán 159,8k So sánh"
2,333147,Điện thoại Xiaomi Redmi Note 14 8GB/128GB,5490000.0,Xiaomi,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/333147/...,"Full HD+ 6.67""",Chip MediaTek Helio G99-Ultra 8 nhân RAM: 8 GB...,"4.9 • Đã bán 78,9k So sánh"
3,332936,Điện thoại OPPO Reno13 F 5G 8GB/256GB,9990000.0,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/332936/...,"Full HD+ 6.67""",Chip Snapdragon 6 Gen 1 5G 8 nhân RAM: 8 GB Du...,"4.9 • Đã bán 66,3k So sánh"
4,335755,Điện thoại Xiaomi Redmi 13x 6GB/128GB,4290000.0,Xiaomi,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/335755/...,"Full HD+ 6.79""",Chip MediaTek Helio G91 Ultra 8 nhân RAM: 6 GB...,"5 • Đã bán 15,2k So sánh"
...,...,...,...,...,...,...,...,...,...
118,319904,Điện thoại Samsung Galaxy A25 5G 8GB/128GB,6490000.0,Samsung,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/319904/...,"Full HD+ 6.5""",Chip Exynos 1280 RAM: 8 GB Dung lượng: 128 GB ...,"4.9 • Đã bán 79,4k So sánh"
119,320836,Điện thoại OPPO A38 6GB/128GB,3990000.0,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/320836/...,"HD+ 6.56""",Chip MediaTek Helio G85 RAM: 6 GB Dung lượng: ...,"4.9 • Đã bán 127,7k So sánh"
120,320964,Điện thoại Samsung Galaxy M15 5G 6GB/128GB,4990000.0,Samsung,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/320964/...,"Full HD+ 6.5""",Chip MediaTek Dimensity 6100+ RAM: 6 GB Dung l...,"4.9 • Đã bán 95,5k So sánh"
121,330619,Điện thoại realme 13+ 5G 12GB/256GB,9990000.0,realme,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/330619/...,"Full HD+ 6.67""",Chip MediaTek Dimensity 7300 Energy 5G 8 nhân ...,"4.9 • Đã bán 2,4k So sánh"


## 🛠️ 4. Extract numeric values from 'phone_rating'

In [23]:
# Extract number of votes
def enrich_vote_rating_raw(row):
    _ = row.split()[0]
    return _.replace('•','')
df['vote_rating'] = df['phone_rating'].apply(enrich_vote_rating_raw)

# # Extract sold quantity
def enrich_sold_quantity_raw(row):
    pattern=r'\d{1,3}(?:,\d{1,3})?k?'
    try:
         return re.findall(pattern,row)[-1]
    except:
        return 0
df['phone_rating'].apply(enrich_sold_quantity_raw)

df['sold_quantity'] = df['phone_rating'].apply(enrich_sold_quantity_raw)

# # Replace non-numeric vote ratings with 0
df.loc[df['vote_rating'].str.isalpha(), 'vote_rating'] = 0


## 🛠️ 5. Convert 'sold_quantity' to numeric format

In [24]:
# Handle 'k' (thousands) unit and convert to integer
def enrich_sold_quantity(row):
    if str(row).find('k') > 0:
        _ = int(row.replace(',','').replace('k','')) * 1000
        return _
    else:
        return row
df['sold_quantity']=df['sold_quantity'].apply(enrich_sold_quantity)


## 🛠️ 6. Clean the 'phone_screen' field

In [25]:
# Remove double quotes and clean screen information
def clean_phone_screen(row):
    return row.replace('"','').strip()
df['phone_screen'] = df['phone_screen'].apply(clean_phone_screen)


## 🛠️ 7. Transform data types

In [26]:
# Convert price, sold quantity, and rating to numeric types
df['phone_price'] = df['phone_price'].astype('int64')
df['sold_quantity'] = df['sold_quantity'].astype('int64')
df['vote_rating'] = df['vote_rating'].astype('float64')


## 🛠️ 8. Remove outliers

In [27]:
# Remove products with sold quantity equal to zero
df_2 = df.copy()
df_2 = df_2[df_2['sold_quantity'] > 0]


## 🛠️ 9. Enrich data: calculate gross revenue

In [28]:
# Calculate gross revenue (sold quantity × price)
df_2['gross_revenue'] = df_2['sold_quantity'] * df_2['phone_price']


## 🛠️ 10. Create price segment

In [29]:
# Categorize phones into different price segments
def price_segment(row):
    if row < 10000000:
        return '0 to 10M VND'
    elif row < 20000000:
        return '10M to 20M VND'
    else:
        return 'Upper 20M VND'
df_2['price_segment'] = df_2['phone_price'].apply(price_segment)


## 🛠️ 11. Extract screen size and screen type

In [30]:
# Extract screen size (in inches)
def return_size_screen_phone(row):
    list_row = row.split()
    length_list = len(list_row)
    if length_list < 2:
        return ' '.join(list_row)
    elif length_list < 5:
        return list_row[-1]
    else:
        return ' '.join(list_row[list_row.index('Chính'):])
df_2['size_screen_inch'] = df_2['phone_screen'].apply(return_size_screen_phone)

# Extract screen type
def return_screen_phone(row):
    list_row = row.split()
    length_list = len(list_row)
    if length_list < 2:
        return ' '.join(list_row)
    elif length_list < 5:
        return ' '.join(list_row[:-1])
    else:
        return ' '.join(list_row[:list_row.index('Chính')])

def detail_none_screen(row):
    _ = row.replace('.', '')
    if _.isdigit():
        return 'NaN'
    else:
        return row
df_2['info_screen'] = df_2['phone_screen'].apply(return_screen_phone).apply(detail_none_screen)


## 🛠️ 12. Create rating segment

In [31]:
# Categorize phones based on customer ratings
def rating_segment(row):
    if 0 < row < 3:
        return 'Poor'
    elif 3 < row < 4.5:
        return 'Average'
    else:
        return 'Excellent'
df_2['rating_segment'] = df_2['vote_rating'].apply(rating_segment)


## 🛠️ 13. Extract RAM, ROM, and battery capacity

In [32]:
# Extract RAM and ROM information
def return_ram(row):
    list_row = row.split()
    if 'RAM:' in list_row and 'lượng:' in list_row:
        ram_idx = list_row.index('RAM:')
        return list_row[ram_idx+1]
    else:
        return 'NaN'

def return_rom(row):
    list_row = row.split()
    if 'RAM:' in list_row and 'lượng:' in list_row:
        rom_idx = list_row.index('lượng:')
        return list_row[rom_idx+1]
    else:
        return 'NaN'

df_2['RAM(GB)'] = df_2['phone_utility'].apply(return_ram)
df_2['ROM(GB)'] = df_2['phone_utility'].apply(return_rom)

# Extract battery capacity
def return_battery(row):
    list_row = row.split()
    if 'Pin' in list_row:
        pin_idx = list_row.index('Pin')
        return ' '.join(list_row[pin_idx+1:pin_idx+3]).replace(',', '')
    else:
        return 'NaN'
df_2['battery'] = df_2['phone_utility'].apply(return_battery)


## 🛠️ 14. Create new columns: price in million VND and revenue in billion VND

In [33]:
# Price in Million VND
df_2['phone_price_Milion_VND'] = df_2['phone_price'] / 1_000_000

# Gross revenue in Billion VND
df_2['gross_revenue_Bilion_VND'] = df_2['gross_revenue'] / 1_000_000_000


## 🛠️ 15. Finalize the cleaned dataset

In [34]:
# Drop unnecessary columns
df_3 = df_2.drop(['phone_rating','phone_screen','category','phone_id'], axis=1)

# Reorder columns
df_3 = df_3[['phone_name','phone_brand','phone_price','phone_price_Milion_VND','phone_img','phone_utility','RAM(GB)','ROM(GB)','battery','info_screen','size_screen_inch','vote_rating','price_segment','rating_segment','sold_quantity','gross_revenue','gross_revenue_Bilion_VND']]

# Save cleaned dataset
file = f'Best_sell_phone_cleaned.csv'
df_3.to_csv(file, index=False)


In [36]:
df_3

,phone_name,phone_brand,phone_price,phone_price_Milion_VND,phone_img,phone_utility,RAM(GB),ROM(GB),battery,info_screen,size_screen_inch,vote_rating,price_segment,rating_segment,sold_quantity,gross_revenue,gross_revenue_Bilion_VND
0,Điện thoại OPPO A60 8GB/128GB,OPPO,5490000,5.49,https://cdn.tgdd.vn/Products/Images/42/323543/...,Chip Snapdragon 680 RAM: 8 GB Dung lượng: 128 ...,8,128,5000 mAh,HD+,6.67,4.9,0 to 10M VND,Excellent,2302000,12637980000000,12637.98
1,Điện thoại iPhone 16 Pro Max 256GB,iPhone (Apple),30990000,30.99,https://cdn.tgdd.vn/Products/Images/42/329149/...,Chip Apple A18 Pro 6 nhân RAM: 8 GB Dung lượng...,8,256,33 giờ,Super Retina XDR,6.9,4.9,Upper 20M VND,Excellent,1598000,49522020000000,49522.02
2,Điện thoại Xiaomi Redmi Note 14 8GB/128GB,Xiaomi,5490000,5.49,https://cdn.tgdd.vn/Products/Images/42/333147/...,Chip MediaTek Helio G99-Ultra 8 nhân RAM: 8 GB...,8,128,5500 mAh,Full HD+,6.67,4.9,0 to 10M VND,Excellent,789000,4331610000000,4331.61
3,Điện thoại OPPO Reno13 F 5G 8GB/256GB,OPPO,9990000,9.99,https://cdn.tgdd.vn/Products/Images/42/332936/...,Chip Snapdragon 6 Gen 1 5G 8 nhân RAM: 8 GB Du...,8,256,5800 mAh,Full HD+,6.67,4.9,0 to 10M VND,Excellent,663000,6623370000000,6623.37
4,Điện thoại Xiaomi Redmi 13x 6GB/128GB,Xiaomi,4290000,4.29,https://cdn.tgdd.vn/Products/Images/42/335755/...,Chip MediaTek Helio G91 Ultra 8 nhân RAM: 6 GB...,6,128,5030 mAh,Full HD+,6.79,5.0,0 to 10M VND,Excellent,152000,652080000000,652.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Điện thoại Samsung Galaxy A05s 6GB/128GB,Samsung,3990000,3.99,https://cdn.tgdd.vn/Products/Images/42/317530/...,Chip Snapdragon 680 RAM: 6 GB Dung lượng: 128 ...,6,128,5000 mAh,Full HD+,6.7,4.9,0 to 10M VND,Excellent,1284000,5123160000000,5123.16
118,Điện thoại Samsung Galaxy A25 5G 8GB/128GB,Samsung,6490000,6.49,https://cdn.tgdd.vn/Products/Images/42/319904/...,Chip Exynos 1280 RAM: 8 GB Dung lượng: 128 GB ...,8,128,5000 mAh,Full HD+,6.5,4.9,0 to 10M VND,Excellent,794000,5153060000000,5153.06
119,Điện thoại OPPO A38 6GB/128GB,OPPO,3990000,3.99,https://cdn.tgdd.vn/Products/Images/42/320836/...,Chip MediaTek Helio G85 RAM: 6 GB Dung lượng: ...,6,128,5000 mAh,HD+,6.56,4.9,0 to 10M VND,Excellent,1277000,5095230000000,5095.23
120,Điện thoại Samsung Galaxy M15 5G 6GB/128GB,Samsung,4990000,4.99,https://cdn.tgdd.vn/Products/Images/42/320964/...,Chip MediaTek Dimensity 6100+ RAM: 6 GB Dung l...,6,128,6000 mAh,Full HD+,6.5,4.9,0 to 10M VND,Excellent,955000,4765450000000,4765.45
